In [121]:
#Imports
import pandas as pd
import asyncio
from aiohttp import ClientSession
from typing import Union, List, Dict

#import ollama/Lm studio
from openai import OpenAI
import pickle

In [3]:
#Data 

path = '/Users/mogen/Desktop/Research/df_test_10k.csv'

df_test = pd.read_csv(path)

df_test.head()

,full_text,tweet_id,created_at,screen_name,original_user_id,retweeted_user_ID,collected_at,reply_to_id,reply_to_user,expandedURL
0,"Have a great day Tweeps.. \nPeace &amp; Love, ...",1.676174e+18,2023-07-04 10:19:11+00:00,BellenyFufu,254185636.0,NaN,2023-08-13 13:32:00.832978,NaN,NaN,[]
1,Touching &amp; Inspiring. \n\nThis isn't just ...,1.667404e+18,2023-06-10 05:30:18+00:00,BellenyFufu,254185636.0,NaN,2023-08-13 13:32:00.832997,NaN,NaN,"[{'url': 'https://t.co/xPkuLPXZUu', 'expanded_..."
2,"Follow the value, not the person.",1.664478e+18,2023-06-02 03:45:27+00:00,BellenyFufu,254185636.0,NaN,2023-08-13 13:32:00.833006,NaN,NaN,[]
3,"Now, war is like a hobby, people enjoy killing...",1.661788e+18,2023-05-25 17:34:17+00:00,BellenyFufu,254185636.0,NaN,2023-08-13 13:32:00.833013,NaN,NaN,[]
4,"Have a great day Tweeps.. \nStay safe, stay he...",1.650037e+18,2023-04-23 07:20:57+00:00,BellenyFufu,254185636.0,NaN,2023-08-13 13:32:00.833020,NaN,NaN,[]


In [98]:
# Filter rows where both 'reply_to_id' and 'reply_to_user' are NaN
single_post_df = df_test[df_test[['reply_to_id', 'reply_to_user']].isna().all(axis=1)]
print(len(single_post_df))

reply_post_df = df_test[df_test[['reply_to_id', 'reply_to_user']].notna().any(axis=1)]
print(len(reply_post_df))



4376
5624


In [4]:
# Anzahl der Nutzer
user_counts = df_test.groupby('original_user_id').nunique()
print(f'Found individual {len(user_counts)}')

# Beiträge pro Nutzer
posts_per_user = df_test['original_user_id'].value_counts()
print("Nachrichten pro Top-10-Nutzer:")
print(posts_per_user.iloc[:10])

# Durchschnittliche Anzahl von Nachrichten pro Nutzer
total_messages = len(df_test)
unique_users = df_test['original_user_id'].nunique()
abg_post_p_user = total_messages / unique_users if unique_users > 0 else 0
print(f"Avg per User: {abg_post_p_user:.2f}")

print(f'Found individual {len(user_counts)}')


Found individual 23
Nachrichten pro Top-10-Nutzer:
original_user_id
1.825410e+07    500
1.524432e+18    500
1.552055e+18    500
7.705954e+07    500
1.114290e+18    500
4.144384e+08    500
7.544216e+17    500
1.603207e+18    500
8.341913e+17    500
2.923588e+07    500
Name: count, dtype: int64
Avg per User: 434.78
Found individual 23


In [5]:
# Step 1: Identify starting points
df_test = df_test.copy()  # Ensure you're working with a copy
df_test.loc[:, 'is_start'] = df_test['reply_to_id'].isna()

# Step 1.1: Check if original_user_id with NaN in reply_to_id exists in any other row of reply_to_id
nan_users = df_test.loc[df_test['reply_to_id'].isna(), 'original_user_id']
nan_users_in_replies = nan_users[nan_users.isin(df_test['reply_to_id'])]
if not nan_users_in_replies.empty:
    print(f"Original user IDs with NaN in reply_to_id found in other rows of reply_to_id: {nan_users_in_replies.tolist()}")
else:
    print("No original user IDs with NaN in reply_to_id are found in other rows of reply_to_id.")

# Step 2: Optimize conversation logic for large datasets
conversation_map = {}  # Maps tweet_id to its conversation ID
current_conversation_id = 0

# Use iterrows sparingly, use chunk-based processing to avoid memory issues
chunk_size = 100000  # Define chunk size for processing
for start in range(0, len(df_test), chunk_size):
    chunk = df_test.iloc[start:start+chunk_size]
    for index, row in chunk.iterrows():
        if row['is_start']:
            # Assign a new conversation ID to starting messages
            current_conversation_id += 1
            conversation_map[row['tweet_id']] = current_conversation_id
        else:
            # Assign the conversation ID of the message being replied to
            reply_to = row['reply_to_id']
            if reply_to in conversation_map:
                conversation_map[row['tweet_id']] = conversation_map[reply_to]
            else:
                # If reply_to_id is not found, treat as a new conversation (rare edge case)
                current_conversation_id += 1
                conversation_map[row['tweet_id']] = current_conversation_id

# Step 3: Assign conversation IDs in chunks
conversation_id_chunks = []
for start in range(0, len(df_test), chunk_size):
    chunk = df_test.iloc[start:start+chunk_size].copy()
    chunk['conversation_id'] = chunk['tweet_id'].map(conversation_map)
    conversation_id_chunks.append(chunk)

# Concatenate processed chunks
df_test = pd.concat(conversation_id_chunks, ignore_index=True)

# Debug: Count unmatched reply_to_id
unmatched_replies = df_test[~df_test['reply_to_id'].isin(df_test['tweet_id']) & df_test['reply_to_id'].notna()]
print(f"Unmatched reply_to_id count: {len(unmatched_replies)}")

# Step 4: Group and store conversations in chunks
def group_conversations(chunk):
    grouped = chunk.groupby('conversation_id')
    results = {}
    for conv_id, group in grouped:
        results[conv_id] = {
            'messages': list(group['full_text']),
            'user_ids': list(group['original_user_id']),
            'start_time': group['created_at'].min(),
            'end_time': group['created_at'].max(),
        }
    return results



conversation_chunks = []
for start in range(0, len(df_test), chunk_size):
    chunk = df_test.iloc[start:start+chunk_size]
    conversation_chunks.append(group_conversations(chunk))

# Merge all conversation chunks
conversations = {}
for conv in conversation_chunks:
    conversations.update(conv)

# Step 5: Create a DataFrame and drop rows with only one message
conversation_df = pd.DataFrame.from_dict(conversations, orient='index')
conversation_df.reset_index(inplace=True)
conversation_df.rename(columns={'index': 'conversation_id'}, inplace=True)

# Filter out rows with only one message
conversation_df = conversation_df[conversation_df['messages'].apply(len) > 1]

# Count NaN values in a given column
def count_nan(column_name):
    return df_test[column_name].isna().sum()



No original user IDs with NaN in reply_to_id are found in other rows of reply_to_id.
Unmatched reply_to_id count: 5261


In [99]:
#Create  posts lists 

# New method: Get all messages for each user
def get_user_messages(df):
    user_messages = df.groupby('original_user_id').apply(
        lambda x: {
            'original_user_id':	set(list(x['original_user_id'])),
            'full_text':list(x['full_text']),
            'tweet_id':list(x['tweet_id']),
            'created_at':	list(x['created_at']),
            'screen_name':	set(list(x['screen_name'])),
            'retweeted_user_ID':	list(x['retweeted_user_ID']),
            'collected_at':	list(x['collected_at']),
            'reply_to_id':	list(x['reply_to_id']),
            'reply_to_user':	list(x['reply_to_user']),
            'expandedURL':list(x['expandedURL']),
            
        }
    ).to_dict()
    return user_messages

# Example usage
user_messages = get_user_messages(single_post_df)


/var/folders/gm/t6hjnb8x6qq26hlg3mq4p9vc0000gn/T/ipykernel_23999/3343922282.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  user_messages = df.groupby('original_user_id').apply(


In [100]:
def describe_dict_structure(d, indent=0):
    """
    Recursively prints the structure of a dictionary, showing its keys and types.

    :param d: The dictionary to analyze.
    :param indent: Current indentation level for nested structures.
    """
    if not isinstance(d, dict):
        print(" " * indent + f"[Non-dict object of type {type(d).__name__}]")
        return

    for key, value in d.items():
        if isinstance(value, dict):
            print(" " * indent + f"{key}: dict")
            describe_dict_structure(value, indent + 4)
        elif isinstance(value, list):
            print(" " * indent + f"{key}: list (len={len(value)})")
            if value and isinstance(value[0], dict):
                print(" " * (indent + 4) + "[Elements are dictionaries]")
                describe_dict_structure(value[0], indent + 8)
        else:
            print(" " * indent + f"{key}: {type(value).__name__}")

describe_dict_structure(user_messages)


18254098.0: dict
    original_user_id: set
    full_text: list (len=90)
    tweet_id: list (len=90)
    created_at: list (len=90)
    screen_name: set
    retweeted_user_ID: list (len=90)
    collected_at: list (len=90)
    reply_to_id: list (len=90)
    reply_to_user: list (len=90)
    expandedURL: list (len=90)
29235878.0: dict
    original_user_id: set
    full_text: list (len=208)
    tweet_id: list (len=208)
    created_at: list (len=208)
    screen_name: set
    retweeted_user_ID: list (len=208)
    collected_at: list (len=208)
    reply_to_id: list (len=208)
    reply_to_user: list (len=208)
    expandedURL: list (len=208)
77059536.0: dict
    original_user_id: set
    full_text: list (len=476)
    tweet_id: list (len=476)
    created_at: list (len=476)
    screen_name: set
    retweeted_user_ID: list (len=476)
    collected_at: list (len=476)
    reply_to_id: list (len=476)
    reply_to_user: list (len=476)
    expandedURL: list (len=476)
254185636.0: dict
    original_user_id:

In [101]:
prompt_persona_analyze = '''Please analyze the following social media posts to produce a detailed and comprehensive character description of the user. Delve deeply into the user's personality traits, going beyond standard models like the Big Five to include a wide range of characteristics observable through social media behavior. Be creative and consider aspects such as:

Humor style
Communication patterns
Emotional expressions
Values and beliefs
Interests and hobbies
Social interactions and relationships
Coping mechanisms and stress responses
Moral and ethical viewpoints
Ambitions and life goals
Cultural and societal influences
Additionally, identify any political and social associations, affiliations, or leanings evident from the posts. Estimate basic demographic information such as age, sex, cultural background, and any other relevant traits, supporting your estimations with evidence from the content.

Focus on providing a rich, narrative description that encapsulates the user's character as revealed through their social media activity. Base your analysis solely on the content provided, avoiding unwarranted assumptions.

'''

In [102]:
df_new = pd.json_normalize(
    [
        {**{'user_id': k}, **v} 
        for k, v in user_messages.items()
    ]
)

df_new

,user_id,original_user_id,full_text,tweet_id,created_at,screen_name,retweeted_user_ID,collected_at,reply_to_id,reply_to_user,expandedURL
0,1.825410e+07,{18254098.0},[Best Buy openly discriminates against white p...,"[1.6904777818792755e+18, 1.6904752322499297e+1...","[2023-08-12 21:38:05+00:00, 2023-08-12 21:27:5...",{KurtSchemers},"[1.3262297375519127e+18, 39844876.0, 4239551.0...","[2023-08-13 13:32:09.303981, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [], [], [], [], [{'url': 'https://t.co/jO..."
1,2.923588e+07,{29235878.0},"[Okay, Republicans, we women have about had it...","[1.6907177456990454e+18, 1.6907167987384484e+1...","[2023-08-13 13:31:37+00:00, 2023-08-13 13:27:5...",{ZagoZana},"[9.007336687795444e+17, 42518865.0, nan, 1.567...","[2023-08-13 13:32:07.525988, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [], [], [], [], [{'url': 'https://t.co/NC..."
2,7.705954e+07,{77059536.0},"[At the ISPC Nationals, @coley_shane was Stand...","[1.5745019497262449e+18, 1.5707746601024266e+1...","[2022-09-26 20:51:32+00:00, 2022-09-16 14:00:3...",{GLOCKInc},"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2023-08-13 13:32:15.374826, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [{'url': 'https://t.co/GEhWO5EI1a', 'expa..."
3,2.541856e+08,{254185636.0},"[Have a great day Tweeps.. \nPeace &amp; Love,...","[1.676173805482553e+18, 1.6674037971808584e+18...","[2023-07-04 10:19:11+00:00, 2023-06-10 05:30:1...",{BellenyFufu},"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2023-08-13 13:32:00.832978, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [{'url': 'https://t.co/xPkuLPXZUu', 'expa..."
4,4.144384e+08,{414438377.0},"[@ChrisMurphyCT https://t.co/6a3CUHQx3h, @JoeB...","[1.6906218249529057e+18, 1.6906213321054003e+1...","[2023-08-13 07:10:27+00:00, 2023-08-13 07:08:3...",{Craigar48},"[1.519400530258375e+18, 1.508953280793256e+18,...","[2023-08-13 13:32:13.490662, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [], [], [], [], [{'url': 'https://t.co/gA..."
5,1.495726e+09,{1495726466.0},[What do you get if you cross the UK comedy sc...,"[1.690639914721833e+18, 1.690615418744955e+18,...","[2023-08-13 08:22:20+00:00, 2023-08-13 06:45:0...",{KonstantinKisin},"[9.757802149188362e+17, 1.0321795130886513e+18...","[2023-08-13 13:32:01.200189, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [], [], [], [], [], [{'url': 'https://t.c..."
6,7.544216e+17,{7.544215523518505e+17},"[https://t.co/R6VwgNAiBO, The GOP has the ""bes...","[1.6905790470912287e+18, 1.690535546437722e+18...","[2023-08-13 04:20:28+00:00, 2023-08-13 01:27:3...",{OvereducatedFem},"[1.3931934452466606e+18, nan, 1.25630449225498...","[2023-08-13 13:32:11.885122, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [{'url': 'https://t.co/ZkaXraU4Qy', 'expa..."
7,8.251444e+17,{8.251444442989978e+17},"[On average, you need to make 5 sales calls be...","[1.6907013930136658e+18, 1.6906945034876357e+1...","[2023-08-13 12:26:38+00:00, 2023-08-13 11:59:1...",{CallahanAutoCo},"[nan, nan, nan, nan, nan, 8.251444442989978e+1...","[2023-08-13 13:32:20.885620, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [], [], [], [{'url': 'https://t.co/diB0DV..."
8,8.341913e+17,{8.341912590570004e+17},[How the hell is this moron the leading Republ...,"[1.690714816032166e+18, 1.6907147523291013e+18...","[2023-08-13 13:19:58+00:00, 20

#### Plain-Formatter

returns a String of the following format: 

    Instruction (prompt_persona_analyze)

    User's Tweets: {full_text}
    Author: {screen_name}

In [112]:
def create_detailed_prompt(row):
    # Handle full_text list
    if isinstance(row['full_text'], list):
        # Take first 50 tweets or all if less available
        tweets_to_use = row['full_text'][:50]
        full_text = "\n".join(f"Tweet {i+1}: {tweet}" for i, tweet in enumerate(tweets_to_use))
        total_tweets = len(row['full_text'])
        #print(f"{total_tweets} total tweets\n")
    else:
        full_text = str(row['full_text'])
    
    screen_names = str(row['screen_name'])
    
    return f"""
User's Tweets: {full_text}
Author: {screen_names}
"""


# Create prompts with error handling
prompts = []
for idx, row in df_new.iterrows():
    try:
        user_content = create_detailed_prompt(row)
        messages = prompt_persona_analyze + '\n' +user_content
        prompts.append(messages)
    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        print("Row content:", row)
#append prompt to Df for later
df_new['prompt']=prompts



Please analyze the following social media posts to produce a detailed and comprehensive character description of the user. Delve deeply into the user's personality traits, going beyond standard models like the Big Five to include a wide range of characteristics observable through social media behavior. Be creative and consider aspects such as:

Humor style
Communication patterns
Emotional expressions
Values and beliefs
Interests and hobbies
Social interactions and relationships
Coping mechanisms and stress responses
Moral and ethical viewpoints
Ambitions and life goals
Cultural and societal influences
Additionally, identify any political and social associations, affiliations, or leanings evident from the posts. Estimate basic demographic information such as age, sex, cultural background, and any other relevant traits, supporting your estimations with evidence from the content.

Focus on providing a rich, narrative description that encapsulates the user's character as revealed through t

### LLama

#### LLama Prompt Formatter

In [118]:
class Llama3DataFormatter:
    """
    A utility class for formatting prompts according to the Llama 3 Instruct specification,
    with specific support for processing pandas DataFrames.
    """
    
    # Special tokens
    BEGIN_TEXT = "<|begin_of_text|>"
    START_HEADER = "<|start_header_id|>"
    END_HEADER = "<|end_header_id|>"
    EOT = "<|eot_id|>"
    
    @staticmethod
    def format_message(role: str, content: str) -> str:
        """Format a single message with its role."""
        return f"{Llama3DataFormatter.START_HEADER}{role}{Llama3DataFormatter.END_HEADER}\n\n{content}{Llama3DataFormatter.EOT}"
    
    @staticmethod
    def format_prompt(messages: List[Dict[str, str]], system_message: str = None) -> str:
        """
        Format a complete prompt with optional system message and conversation history.
        """
        prompt = [Llama3DataFormatter.BEGIN_TEXT]
        
        # Add system message if provided
        if system_message:
            prompt.append(Llama3DataFormatter.format_message("system", system_message))
        
        # Add conversation messages
        for msg in messages:
            prompt.append(Llama3DataFormatter.format_message(msg['role'], msg['content']))
        
        # Add final assistant header to prompt completion
        prompt.append(f"{Llama3DataFormatter.START_HEADER}assistant{Llama3DataFormatter.END_HEADER}")
        
        return "".join(prompt)
    
    @staticmethod
    def format_two_strings(system_message: str, user_message: str) -> str:
        """
        Create a formatted prompt using only two strings: a system message and a user message.
        """
        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ]
        return Llama3DataFormatter.format_prompt(messages)

    @staticmethod
    def process_dataframe(df: pd.DataFrame, create_detailed_prompt, prompt_persona_analyze: str) -> List[str]:
        """
        Process an entire DataFrame and create formatted prompts for each row.
        
        Args:
            df: pandas DataFrame containing conversation data
            create_detailed_prompt: Function to create user content from row data
            prompt_persona_analyze: System message to use for all prompts
            
        Returns:
            List of formatted prompts
        """
        formatted_prompts = []
        
        for idx, row in df.iterrows():
            try:
                # Create user content using the provided function
                user_content = create_detailed_prompt(row)
                
                # Create messages list similar to your original format
                messages = [
                    {"role": "system", "content": prompt_persona_analyze},
                    {"role": "user", "content": user_content}
                ]
                
                # Format the prompt using Llama 3 format
                formatted_prompt = Llama3DataFormatter.format_prompt(messages)
                formatted_prompts.append(formatted_prompt)
                
            except Exception as e:
                print(f"Error processing row {idx}: {e}")
                print("Row content:", row)
                continue
        
        return formatted_prompts

# Example usage
def example_usage(df, create_detailed_prompt, prompt_persona_analyze):
    formatter = Llama3DataFormatter()
    
    # Process the DataFrame
    formatted_prompts = formatter.process_dataframe(
        df=df,
        create_detailed_prompt=create_detailed_prompt,
        prompt_persona_analyze=prompt_persona_analyze
    )
    
    # Print sample prompts for verification
    for prompt in formatted_prompts[:2]:  # Show first two prompts
        print("\nFormatted Prompt:")
        print(prompt)
        print("\nLength:", len(prompt))
    
    return formatted_prompts

formatter = Llama3DataFormatter()
formatted_prompts = formatter.process_dataframe(
    df=df_new,
    create_detailed_prompt=create_detailed_prompt,
    prompt_persona_analyze=prompt_persona_analyze
)

# Check the results
for prompt in formatted_prompts[:2]:
    print(prompt)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please analyze the following social media posts to produce a detailed and comprehensive character description of the user. Delve deeply into the user's personality traits, going beyond standard models like the Big Five to include a wide range of characteristics observable through social media behavior. Be creative and consider aspects such as:

Humor style
Communication patterns
Emotional expressions
Values and beliefs
Interests and hobbies
Social interactions and relationships
Coping mechanisms and stress responses
Moral and ethical viewpoints
Ambitions and life goals
Cultural and societal influences
Additionally, identify any political and social associations, affiliations, or leanings evident from the posts. Estimate basic demographic information such as age, sex, cultural background, and any other relevant traits, supporting your estimations with evidence from the content.

Focus on providing a rich, narrative description

In [ ]:
# Point to the local server Llama 3.1 8b in 4GGUF os viel zu schawch und produziert IMMER nonsense 

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def persona_creation_call(prompt: str):

  completion = client.chat.completions.create(
    model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
    messages=prompt,
    temperature=0.3,
    seed=420,
    max_tokens=8000,
    #stream = True
  )
  
  return completion

personas=[]
for prompt in prompts[:1]:
    persona=persona_creation_call(prompt)
    personas.append(persona)

   

In [ ]:
df_new['persona']= personas

#### Here LLM call

In [87]:
### as a placeholder the output from Gpt4o
#Author: {'KurtSchemers'}
#prompts[0]

persona= '''
Personality Traits
Humor Style: The user’s humor is heavily sardonic, often expressed through sarcasm, mocking emojis, and rhetorical jabs. They use humor as a tool to ridicule political figures, ideologies, or groups they disagree with, showing a penchant for sharp, pointed commentary rather than lighthearted banter.

Communication Patterns: Their communication is confrontational and assertive, frequently addressing controversial topics with unambiguous language. They often employ rhetorical questions, exclamations, and direct challenges to others, reflecting a combative and unapologetic style. Their posts are rich with hashtags, links, and retweets, emphasizing their intent to amplify messages and engage a like-minded audience.

Emotional Expressions: Emotions conveyed in the posts are intense and polarized, with a strong inclination toward indignation, frustration, and disdain for opposing viewpoints. While negative emotions dominate, there are occasional glimpses of hope or idealism tied to their political aspirations (e.g., their belief in a better future tied to Trump’s re-election).

Values and Beliefs
Political Orientation: The user’s posts reveal a right-leaning, strongly conservative ideology, emphasizing loyalty to Donald Trump, criticism of leftist policies, and skepticism toward mainstream media. They appear to prioritize traditional values, constitutional rights, and a disdain for perceived government overreach, as evidenced by their views on voter fraud, civil asset forfeiture, and perceived political persecution.

Moral and Ethical Viewpoints: The user often frames their arguments within a moral dichotomy of good versus evil, portraying their side as upholding integrity while accusing others of corruption, degeneracy, and hypocrisy. They demonstrate a deep concern for accountability, transparency, and justice but apply these values selectively toward their ideological opponents.

Interests and Hobbies
Political Activism: Politics is the user’s predominant interest, and their posts reflect a high level of engagement with current events, legislative developments, and political commentary. They actively promote content aligned with their ideology and criticize opposing viewpoints with vigor.

Media Consumption: Their engagement with platforms like Fox News, Citizen Free Press, and other conservative-leaning outlets suggests a media diet focused on reinforcing their worldview.

Social Media Advocacy: The user utilizes social media as both a battleground for debate and a platform for advocacy. They are keen on exposing perceived injustices and mobilizing others to challenge entities like Best Buy or governmental agencies.

Social Interactions and Relationships
Style of Engagement: The user interacts primarily with political figures, commentators, and ideologues, often tagging them directly in posts. They display a readiness to debate, albeit with a dismissive tone toward opposing views. Frequent use of “mute” or “block” language suggests they disengage when discussions fail to align with their goals.

Community Dynamics: Their posts cater to a specific in-group of like-minded individuals, fostering a sense of camaraderie through shared grievances and goals. Conversely, they view the out-group (leftists, liberals, or perceived "establishment" figures) with hostility and suspicion.

Coping Mechanisms and Stress Responses
The user demonstrates a preference for outwardly channeling their frustrations through pointed critiques and social media activism. They appear to derive a sense of control and catharsis from exposing and challenging those they perceive as threats. Their dismissive tone toward adversaries and reliance on humor suggest a coping mechanism to deflect or diminish stress.

Ambitions and Life Goals
The user appears to aspire to a societal or political environment aligned with their values—characterized by what they view as justice, transparency, and traditionalism. Their consistent focus on political outcomes and systemic accountability indicates a long-term goal of influencing or contributing to such change, at least within the realm of public opinion.

Cultural and Societal Influences
Cultural Background: Their posts suggest a likely affiliation with mainstream American conservative culture, emphasizing values like patriotism, freedom, and limited government. The consistent reference to American political events and conservative media reinforces this cultural identification.

Societal Context: Their worldview is shaped by a perceived cultural and political battle, with their identity strongly tied to opposition against what they describe as leftist overreach, censorship, and societal degeneration.

Demographics
Age: Likely middle-aged (30–50), inferred from their level of political engagement, familiarity with historical political events, and the tone of their humor.

Sex: The tone and focus of their posts lean toward a traditionally masculine communication style, with aggressive language and themes of dominance. This, coupled with their expressed concerns for issues affecting "white men," suggests they are likely male.

Cultural Background: Predominantly Western, specifically American, with a deep engagement in U.S. political discourse.

Concluding Profile
The user embodies a combative, politically active personality, deeply invested in conservative ideology. Their social media presence is a mix of advocacy, critique, and emotional expression, aiming to influence discourse and rally support for their worldview. They are sharp, confrontational, and unyielding, engaging in debates with a sense of moral certainty. Through their posts, they project an image of resilience and defiance, standing firmly against what they see as a tide of societal and political decay.
'''

### Describe next Posts in a Neutral way 

In [136]:
#
prompt_neutral_post_desc = ''' Create a neutral, high-level description of a social media post.
Focus on the type of content, general topic, and format (e.g., text, image, video).
Avoid including specific quotes, subjective interpretations, or unnecessary details. Keep the description clear, concise, and unbiased.
'''

In [137]:
df_new.head()

,user_id,original_user_id,full_text,tweet_id,created_at,screen_name,retweeted_user_ID,collected_at,reply_to_id,reply_to_user,expandedURL,prompt
0,18254098.0,{18254098.0},[Best Buy openly discriminates against white p...,"[1.6904777818792755e+18, 1.6904752322499297e+1...","[2023-08-12 21:38:05+00:00, 2023-08-12 21:27:5...",{KurtSchemers},"[1.3262297375519127e+18, 39844876.0, 4239551.0...","[2023-08-13 13:32:09.303981, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [], [], [], [], [{'url': 'https://t.co/jO...",Please analyze the following social media post...
1,29235878.0,{29235878.0},"[Okay, Republicans, we women have about had it...","[1.6907177456990454e+18, 1.6907167987384484e+1...","[2023-08-13 13:31:37+00:00, 2023-08-13 13:27:5...",{ZagoZana},"[9.007336687795444e+17, 42518865.0, nan, 1.567...","[2023-08-13 13:32:07.525988, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [], [], [], [], [{'url': 'https://t.co/NC...",Please analyze the following social media post...
2,77059536.0,{77059536.0},"[At the ISPC Nationals, @coley_shane was Stand...","[1.5745019497262449e+18, 1.5707746601024266e+1...","[2022-09-26 20:51:32+00:00, 2022-09-16 14:00:3...",{GLOCKInc},"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2023-08-13 13:32:15.374826, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [{'url': 'https://t.co/GEhWO5EI1a', 'expa...",Please analyze the following social media post...
3,254185636.0,{254185636.0},"[Have a great day Tweeps.. \nPeace &amp; Love,...","[1.676173805482553e+18, 1.6674037971808584e+18...","[2023-07-04 10:19:11+00:00, 2023-06-10 05:30:1...",{BellenyFufu},"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[2023-08-13 13:32:00.832978, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [{'url': 'https://t.co/xPkuLPXZUu', 'expa...",Please analyze the following social media post...
4,414438377.0,{414438377.0},"[@ChrisMurphyCT https://t.co/6a3CUHQx3h, @JoeB...","[1.6906218249529057e+18, 1.6906213321054003e+1...","[2023-08-13 07:10:27+00:00, 2023-08-13 07:08:3...",{Craigar48},"[1.519400530258375e+18, 1.508953280793256e+18,...","[2023-08-13 13:32:13.490662, 2023-08-13 13:32:...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[[], [], [], [], [], [{'url': 'https://t.co/gA...",Please analyze the following social media post...


In [138]:
# Function for asynchronous persona creation call with retry logic
async def post_desc_call_async(session: ClientSession, prompt: str, base_url: str, retries: int = 3):
    """Make asynchronous calls to the LLM with retry logic."""
    for attempt in range(1, retries + 1):
        try:
            async with session.post(
                f"{base_url}/chat/completions",
                json={
                    "model": "lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
                    "messages": prompt,
                    "temperature": 0.3,
                    "seed": 420,
                    "max_tokens": 8000
                },
            ) as response:
                if response.status == 200:
                    result = await response.json()
                    return result.get("choices", [{}])[0].get("message", {}).get("content", "Error: No response")
                else:
                    print(f"Attempt {attempt} failed with status {response.status}: {await response.text()}")
        except Exception as e:
            print(f"Attempt {attempt} failed with exception: {e}")
        
        # Wait before retrying
        await asyncio.sleep(2 ** attempt)  # Exponential backoff
    
    return "Error: Max retries reached"

# Async function to process the DataFrame
async def process_full_text_async(df, prompt_neutral_post_desc, base_url):
    """Process DataFrame rows asynchronously."""
    results = []
    
    # Create an async session
    async with ClientSession() as session:
        tasks = []
        
        for index, row in df.iterrows():
            # Check if 'full_text' has more than 50 entries
            full_text = row['full_text']
            if len(full_text) > 50:
                # Extract the next 10 entries (51-60)
                additional_texts = full_text[50:60]
                
                for text in additional_texts:
                    # Format the prompt
                    text_content = 'Describe the following Twitter Post in a neutral way: ' + text
                    prompt = Llama3DataFormatter.format_two_strings(prompt_neutral_post_desc, text_content)
                    
                    # Append the task for async processing
                    tasks.append(
                        post_desc_call_async(session, prompt, base_url)
                    )
                    
                    # Append row metadata to results for matching
                    results.append({
                        "user_id": row['original_user_id'],
                        "original_text": text_content,
                        "llm_response": None  # Placeholder, to be updated later
                    })
        
        # Execute all tasks concurrently
        responses = await asyncio.gather(*tasks)
        
        # Update results with responses
        for i, response in enumerate(responses):
            results[i]["llm_response"] = response
    
    return pd.DataFrame(results)

# Async driver function
async def main():
    # Replace `df_new` and `prompt_neutral_post_desc` with your actual variables
    prompt_neutral_post_desc = "Analyze and neutrally describe the provided post."
    base_url = "http://localhost:1234/v1"  # LLM server base URL

    # Process the DataFrame asynchronously
    results_df = await process_full_text_async(df_new, prompt_neutral_post_desc, base_url)
    
    # Save the results
    results_df.to_parquet('/Users/mogen/Desktop/Research/storage/neutral_posts_desc.parquet')
    print("Results saved!")
    
# Run the async function
nest_asyncio.apply()
await main()


Attempt 1 failed with status 400: {"error":"'messages' field must be either an array of objects with 'role' and 'content' fields, or a string that can be parsed as JSON into an array of objects with 'role' and 'content' fields."}
Attempt 1 failed with status 400: {"error":"'messages' field must be either an array of objects with 'role' and 'content' fields, or a string that can be parsed as JSON into an array of objects with 'role' and 'content' fields."}
Attempt 1 failed with status 400: {"error":"'messages' field must be either an array of objects with 'role' and 'content' fields, or a string that can be parsed as JSON into an array of objects with 'role' and 'content' fields."}
Attempt 1 failed with status 400: {"error":"'messages' field must be either an array of objects with 'role' and 'content' fields, or a string that can be parsed as JSON into an array of objects with 'role' and 'content' fields."}
Attempt 1 failed with status 400: {"error":"'messages' field must be either an a

In [128]:
results_df = pd.read_parquet('/Users/mogen/Desktop/Research/storage/neutral_posts_desc.parquet')
results_df

,user_id,original_text,llm_response
0,[18254098.0],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...
1,[18254098.0],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...
2,[18254098.0],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...
3,[18254098.0],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...
4,[18254098.0],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...
...,...,...,...
176,[1.6126174631196344e+18],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...
177,[1.6126174631196344e+18],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...
178,[1.6126174631196344e+18],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...
179,[1.6154129375433236e+18],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...


### Combine and Format for synthetic post generation 

In [135]:
# Initialize lists to store the prompts and personas
prompt_formatted = []
personas_ = []

# Iterate over the rows of results_df
for idx, row in results_df.iterrows():
    user_id = row['user_id']
    user_id = user_id[0]
    print(user_id)
    # Check if the user ID exists in 'original_user_id' column
    if user_id in df_new['user_id'].values:
        # Retrieve the corresponding 'personsa' value
        personsa_value = df_new.loc[df_new['original_user_id'] == user_id, 'personsa'].iloc[0]
        
        Instruct = f'''Using the detailed character analysis provided, generate responses and make decisions as if you are the user described.
Your goal is to authentically replicate the user's beliefs, attitudes, and decision-making processes in new, unseen scenarios.
Persona: {personsa_value}'''
        
        # Retrieve the 'llm_response' for the current user
        scenario = row['llm_response']
        
        # Format the prompt using Llama3DataFormatter
        formatted_prompt = Llama3DataFormatter.format_two_strings(Instruct, scenario)
        
        # Append the formatted prompt and persona to the lists
        prompt_formatted.append(formatted_prompt)
        personas_.append(personsa_value)
    else:
        # Handle the case where the user ID is not found
        personas_.append('NaN')
        prompt_formatted.append('User ID not found.')
        print(f"User ID {user_id} not found.")

# Add the new columns to results_df
results_df['System_prompt'] = personas_
results_df['Formatted_Prompt'] = prompt_formatted


18254098.0


KeyError: 'personsa'

In [133]:
results_df

,user_id,original_text,llm_response,System_prompt,Formatted_Prompt
0,[18254098.0],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...,NaN,User ID not found.
1,[18254098.0],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...,NaN,User ID not found.
2,[18254098.0],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...,NaN,User ID not found.
3,[18254098.0],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...,NaN,User ID not found.
4,[18254098.0],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...,NaN,User ID not found.
...,...,...,...,...,...
176,[1.6126174631196344e+18],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...,NaN,User ID not found.
177,[1.6126174631196344e+18],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...,NaN,User ID not found.
178,[1.6126174631196344e+18],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...,NaN,User ID not found.
179,[1.6154129375433236e+18],Describe the following Twitter Post in a neutr...,Error: Cannot connect to host localhost:1234 s...,NaN,User ID not found.
